In [18]:

from tqdm.auto import tqdm

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import PromptTemplate
import json
import pandas as pd
from pprint import pprint
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# from langchain_openai import OpenAIEmbeddings
# from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
import os
import re
import html
import unicodedata
from typing import Optional, Tuple
os.environ["OPENAI_API_KEY"] = "xxxxxxxx"

C:\Users\fpatr\anaconda3\envs\Topic_modeling\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_logging = pd.read_csv("logging_data_2025.csv")
df_logging

C:\Users\fpatr\AppData\Local\Temp\ipykernel_36428\2926410520.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_logging = pd.read_csv("logging_data_2025.csv")


,Unnamed: 0,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,0,4572616,1,4574910.0,NaN,2010-12-31 22:53:27,NaN,1,937.0,<p>I've read a lot of material today on the to...,...,NaN,2011-01-01 17:28:42,How to log an exception without a controller i...,<logging><error-handling><asp.net-mvc-3>,1.0,0,NaN,NaN,NaN,CC BY-SA 2.5
1,1,4571790,1,NaN,NaN,2010-12-31 18:47:29,NaN,7,2887.0,<p>I just spent the better part of 3 hours try...,...,2011-01-01 22:20:23,2013-07-04 14:06:32,Logging with log4j on tomcat jruby-rack for a ...,<log4j><ruby-on-rails-3><tomcat6><jrubyonrails...,2.0,0,NaN,NaN,NaN,CC BY-SA 2.5
2,2,4571389,1,4571404.0,NaN,2010-12-31 17:21:42,NaN,2,1594.0,<p>Many logging systems allow you to filter th...,...,NaN,2010-12-31 17:29:13,"Is there a standard order for logging levels, ...",<language-agnostic><logging>,2.0,0,NaN,NaN,NaN,CC BY-SA 2.5
3,3,4570072,1,4570095.0,NaN,2010-12-31 12:24:26,NaN,67,104978.0,<p>Suddenly this keeps happening during a JUni...,...,2012-02-28 22:23:11,2016-10-25 04:04:40,How to configure log4j.properties for SpringJU...,<java><spring><log4j><junit4>,7.0,0,NaN,NaN,NaN,CC BY-SA 3.0
4,4,4569687,1,NaN,NaN,2010-12-31 11:04:01,NaN,0,7525.0,<p>So I'm having trouble with log4j and hbm2dd...,...,2010-12-31 12:12:14,2010-12-31 13:23:16,ClassNotFoundException javax.mail.internet.Add...,<log4j><maven><hbm2ddl>,2.0,0,NaN,NaN,NaN,CC BY-SA 2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216089,216089,45181661,1,45402617.0,NaN,2017-07-19 05:30:27,NaN,0,1627.0,<p>In my app.php file:</p>\n\n<pre><code>use M...,...,2017-07-19 12:00:20,2017-07-30 17:21:52,How to use monolog in slim 3?,<monolog><slim-3>,2.0,2,NaN,NaN,NaN,CC BY-SA 3.0
216090,216090,45455261,1,45456455.0,NaN,2017-08-02 08:17:42,NaN,1,5473.0,<p>I'm trying to stream logs from a symfony 2 ...,...,NaN,2019-12-07 22:39:05,Graylog / Symfony2 / Gelf: How to configure lo...,<php><symfony><monolog><graylog2><gelf>,2.0,0,NaN,NaN,NaN,CC BY-SA 3.0
216091,216091,77292842,1,NaN,NaN,2023-10-14 12:45:20,NaN,0,37.0,<p>I currently have this configuration for my ...,...,NaN,2023-10-14 12:45:20,How do you log out winston the same way consol...,<winston>,0.0,0,NaN,NaN,NaN,CC BY-SA 4.0
216092,216092,67812209,1,NaN,NaN,2021-06-02 20:34:40,NaN,0,756.0,<p>I'm using Drupal 8 with the Monolog module ...,...,NaN,2021-08-04 15:52:11,How do I control Drupal 8 monolog dblog log-le...,<drupal-8><monolog>,1.0,0,NaN,NaN,NaN,CC BY-SA 4.0


In [5]:
df_with = pd.read_csv("logging_with_topic.csv")
df_with

,Unnamed: 0,Id,Title,Body,topic_name
0,0,78985681,Use Fluent API to control how complex objects ...,<p>Let's consider i have a model.</p>\n<pre cl...,Custom Logging Frameworks
1,1,78985635,How to configure slf4j and logback to allow on...,<p>I have one Java class that is multithreaded...,Logging Levels and Output
2,2,78985358,log44j2 get string before add it to a file,"<p>I Have Used Log4j2 framework, and it produc...",Custom Logging Frameworks
3,3,78983538,"Set TORCH_LOGS=""+dynamo"" and TORCHDYNAMO_VERBO...",<p>I am trying to use <code>torch.compile</cod...,Error Handling and Debugging
4,4,78983194,Is this a valid approach to use POSIX signals ...,<p>I want to be able to set python logging lev...,Logging Levels and Output
...,...,...,...,...,...
75513,75513,73730972,zerolog with stdout and file logger adds addit...,<p>I am using <code>zerolog</code> package in ...,Logging Levels and Output
75514,75514,76851520,How do I test that zerolog logger raised log e...,<p>In my Go app I moved error logging into a s...,Logging Levels and Output
75515,75515,77823971,Change position level and timestamp zerolog go...,<p>i'm using zerolog for logging and got compl...,Logging Levels and Output
75516,75516,77824746,How to extract the zerolog event fields in the...,<p>I wrote a code for integrating Azure app in...,Custom Logging Frameworks


In [7]:
df_logging = df_logging[["Title", "Body"]]
df_logging

,Title,Body
0,How to log an exception without a controller i...,<p>I've read a lot of material today on the to...
1,Logging with log4j on tomcat jruby-rack for a ...,<p>I just spent the better part of 3 hours try...
2,"Is there a standard order for logging levels, ...",<p>Many logging systems allow you to filter th...
3,How to configure log4j.properties for SpringJU...,<p>Suddenly this keeps happening during a JUni...
4,ClassNotFoundException javax.mail.internet.Add...,<p>So I'm having trouble with log4j and hbm2dd...
...,...,...
216089,How to use monolog in slim 3?,<p>In my app.php file:</p>\n\n<pre><code>use M...
216090,Graylog / Symfony2 / Gelf: How to configure lo...,<p>I'm trying to stream logs from a symfony 2 ...
216091,How do you log out winston the same way consol...,<p>I currently have this configuration for my ...
216092,How do I control Drupal 8 monolog dblog log-le...,<p>I'm using Drupal 8 with the Monolog module ...


In [8]:
def remove_duplicates(df_logging: pd.DataFrame, df_with: pd.DataFrame) -> pd.DataFrame:
    """
    Remove rows from df_logging that have the same Title and Body as in df_with.
    
    Parameters
    ----------
    df_logging : pd.DataFrame
        The main DataFrame containing logging records.
    df_with : pd.DataFrame
        The DataFrame containing rows to exclude.
    
    Returns
    -------
    pd.DataFrame
        A new DataFrame with duplicates removed.
    """
    # Ensure both DataFrames only keep Title and Body for comparison
    df_with_subset = df_with[["Title", "Body"]].drop_duplicates()
    
    # Perform anti-join (keep rows in df_logging not in df_with_subset)
    df_result = df_logging.merge(
        df_with_subset,
        on=["Title", "Body"],
        how="left",
        indicator=True
    )
    
    # Keep only rows not found in df_with
    df_result = df_result[df_result["_merge"] == "left_only"].drop(columns=["_merge"])
    
    return df_result


In [9]:
df_cleaned = remove_duplicates(df_logging, df_with)
df_cleaned.shape

(119124, 2)

In [10]:
df_cleaned

,Title,Body
16,log4net with ASP.NET MVC: nothing happens,<p>I am trying to use log4net with ASP.NET MVC...
22,How can I resolve the event log full error upo...,<p>I want to add a new connection for a SQL da...
23,Customizing NSLog Function on iPhone,<p>I know that it's possible to do method swiz...
26,Why Does My Vector<PEVENTLOGRECORD> Mysterious...,<p>I am making a program that reads and stores...
34,Use log4net in SDK,<p>I am providing an SDK using C#. To enable f...
...,...,...
215257,why isn't my GCS-triggered Python Gen2 Cloud F...,<p>I am trying to trigger a simple Python 3.11...
215258,How do I stop my app from auto-logging my Andr...,<p>I am testing authentication with a mobile a...
215259,LastLoginTime not present in Windows 11. Is th...,<p>I am working to write some simple powershel...
215260,Why is my server not logging a login session?,<p>For some context: I'm trying to make a basi...


In [13]:
_CODE_FENCE_RE = re.compile(r"(^|\n)```.*?\n.*?\n```", re.DOTALL | re.IGNORECASE)
_HTML_CODE_RE  = re.compile(r"(?is)<pre[^>]*>.*?</pre>|<code[^>]*>.*?</code>")
_HTML_TAG_RE   = re.compile(r"(?s)<[^>]+>")
_URL_RE        = re.compile(r"https?://\S+|www\.\S+")
_EMAIL_RE      = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w+\b")
_INLINE_BACKTICK_RE = re.compile(r"`[^`]*`")
_MULTISPACE_RE = re.compile(r"[ \t]+")

def _as_text(x) -> str:
    """Return a clean string; map None/NaN to ''."""
    try:
        if x is None or pd.isna(x):
            return ""
    except Exception:
        pass
    return x if isinstance(x, str) else str(x)

def _strip_code(text: str) -> str:
    if not text:
        return ""
    text = _CODE_FENCE_RE.sub("\n", text)
    text = _INLINE_BACKTICK_RE.sub(" ", text)
    text = _HTML_CODE_RE.sub(" ", text)
    return text

def _strip_html(text: str) -> str:
    if not text:
        return ""
    text = html.unescape(text)
    text = _HTML_TAG_RE.sub(" ", text)
    return text

def _basic_normalize(text: str) -> str:
    if not text:
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = _URL_RE.sub(" ", text)
    text = _EMAIL_RE.sub(" ", text)
    text = _MULTISPACE_RE.sub(" ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = "\n".join(line.strip() for line in text.splitlines())
    return text.strip()

def preprocess_title_body(
    title: Optional[str],
    body: Optional[str],
    *,
    lowercase: bool = False,
    max_chars: Optional[int] = 8000,
) -> Tuple[str, str, str]:
    """Clean a post's title/body for LLM topic classification."""
    # NEW: coerce to strings safely
    t = _as_text(title)
    b = _as_text(body)

    t = _strip_code(t);  b = _strip_code(b)
    t = _strip_html(t);  b = _strip_html(b)
    t = _basic_normalize(t);  b = _basic_normalize(b)

    if lowercase:
        t = t.lower(); b = b.lower()

    for_llm = f"Title:\n{t}\n\nBody:\n{b}".strip()
    if max_chars is not None and len(for_llm) > max_chars:
        for_llm = for_llm[:max_chars].rsplit(" ", 1)[0] + " …"

    return t, b, for_llm

def preprocess_dataframe(
    df: pd.DataFrame,
    title_col: str = "Title",
    body_col: str = "Body",
    *,
    lowercase: bool = False,
    max_chars: Optional[int] = 8000,
) -> pd.DataFrame:
    out = df.copy()
    res = out.apply(
        lambda row: preprocess_title_body(
            row.get(title_col, ""), row.get(body_col, ""),
            lowercase=lowercase, max_chars=max_chars
        ),
        axis=1
    )
    out["clean_title"] = [t for t, _, _ in res]
    out["clean_body"]  = [b for _, b, _ in res]
    out["llm_input"]   = [c for _, _, c in res]
    return out


In [14]:
# For your DataFrame:
df_ready = preprocess_dataframe(df_cleaned, title_col="Title", body_col="Body",
                                lowercase=False, max_chars=8000)

In [15]:

# The column `llm_input` is ready to feed an LLM-as-judge prompt per row.
df_ready[["clean_title", "clean_body", "llm_input"]].head()

,clean_title,clean_body,llm_input
16,log4net with ASP.NET MVC: nothing happens,I am trying to use log4net with ASP.NET MVC an...,Title:\nlog4net with ASP.NET MVC: nothing happ...
22,How can I resolve the event log full error upo...,I want to add a new connection for a SQL datab...,Title:\nHow can I resolve the event log full e...
23,Customizing NSLog Function on iPhone,I know that it's possible to do method swizzli...,Title:\nCustomizing NSLog Function on iPhone\n...
26,Why Does My Vector Mysteriously Get Cleared?,I am making a program that reads and stores da...,Title:\nWhy Does My Vector Mysteriously Get Cl...
34,Use log4net in SDK,I am providing an SDK using C#. To enable fiel...,Title:\nUse log4net in SDK\n\nBody:\nI am prov...


In [16]:
df_ready

,Title,Body,clean_title,clean_body,llm_input
16,log4net with ASP.NET MVC: nothing happens,<p>I am trying to use log4net with ASP.NET MVC...,log4net with ASP.NET MVC: nothing happens,I am trying to use log4net with ASP.NET MVC an...,Title:\nlog4net with ASP.NET MVC: nothing happ...
22,How can I resolve the event log full error upo...,<p>I want to add a new connection for a SQL da...,How can I resolve the event log full error upo...,I want to add a new connection for a SQL datab...,Title:\nHow can I resolve the event log full e...
23,Customizing NSLog Function on iPhone,<p>I know that it's possible to do method swiz...,Customizing NSLog Function on iPhone,I know that it's possible to do method swizzli...,Title:\nCustomizing NSLog Function on iPhone\n...
26,Why Does My Vector<PEVENTLOGRECORD> Mysterious...,<p>I am making a program that reads and stores...,Why Does My Vector Mysteriously Get Cleared?,I am making a program that reads and stores da...,Title:\nWhy Does My Vector Mysteriously Get Cl...
34,Use log4net in SDK,<p>I am providing an SDK using C#. To enable f...,Use log4net in SDK,I am providing an SDK using C#. To enable fiel...,Title:\nUse log4net in SDK\n\nBody:\nI am prov...
...,...,...,...,...,...
215257,why isn't my GCS-triggered Python Gen2 Cloud F...,<p>I am trying to trigger a simple Python 3.11...,why isn't my GCS-triggered Python Gen2 Cloud F...,I am trying to trigger a simple Python 3.11 Ge...,Title:\nwhy isn't my GCS-triggered Python Gen2...
215258,How do I stop my app from auto-logging my Andr...,<p>I am testing authentication with a mobile a...,How do I stop my app from auto-logging my Andr...,I am testing authentication with a mobile app....,Title:\nHow do I stop my app from auto-logging...
215259,LastLoginTime not present in Windows 11. Is th...,<p>I am working to write some simple powershel...,LastLoginTime not present in Windows 11. Is th...,I am working to write some simple powershell s...,Title:\nLastLoginTime not present in Windows 1...
215260,Why is my server not logging a login session?,<p>For some context: I'm trying to make a basi...,Why is my server not logging a login session?,For some context: I'm trying to make a basic l...,Title:\nWhy is my server not logging a login s...


In [17]:
df_ready.to_csv("llm_input.csv")

In [30]:
# !pip install "langchain>=0.2" "langchain-openai>=0.1" pandas tqdm
# -----------------------------
# 1) Prompt (plain string)
# -----------------------------
# -----------------------------
# 1) Prompt (uses one input: {text})
# -----------------------------
PROMPT = """You are an expert in logging practices and software engineering.
Classify the following Stack Overflow post (already preprocessed as Title/Body text) into one of the predefined topics.
If no topic matches well, propose a new topic name.
Also extract 3–7 important keywords that justify your choice.

Return JSON only with keys: "topic" and "keywords".

Available Topics and Descriptions:
1. Logging Levels and Output – Problems related to choosing log levels (INFO, DEBUG, ERROR, etc.), formatting log messages, and configuring log outputs.
2. General Logging Practices – Questions about the general use of logging across languages and platforms (Python, Java, .NET, Android, etc.).
3. Error Handling and Debugging – Issues where logging is connected to debugging errors, crashes, or error traces.
4. File Logging and Configuration – Managing, configuring, and troubleshooting log files (rotation, paths, archiving).
5. Custom Logging Frameworks – Building/extending custom logging systems, or integrating specialized logging into frameworks.
6. Data and Tensorboard Logging – Logging in data/ML contexts (TensorBoard, MLflow, DVC, model metrics).
7. Logback and Application Configuration – Configuring/troubleshooting Java Logback (often Spring Boot).
8. Logging in Containerized Environments – Logging with Docker, Kubernetes, container runtimes and collectors.
9. NLog and .NET Logging – Using or configuring NLog in .NET applications.
10. Testing and CI Pipelines – Logging within automated tests or CI/CD (Selenium, Jest, Azure DevOps, etc.).
11. Event Logging and Monitoring – Event logs for monitoring/observability/fault diagnosis (agents, event IDs, log analytics).

Output strictly valid JSON.

Post:

{text}

"""

In [31]:
# -----------------------------
# 2) LLM + prompt
# -----------------------------
# os.environ["OPENAI_API_KEY"] = "sk-..."  # make sure it's set
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
prompt = ChatPromptTemplate.from_template(PROMPT)
chain = prompt | llm

# -----------------------------
# 3) Minimal helpers
# -----------------------------
def _safe_json_loads(s: str):
    """Parse JSON; lightly repair if wrapped in extra text or code fences."""
    try:
        return json.loads(s)
    except Exception:
        if "{" in s and "}" in s:
            s2 = s[s.find("{") : s.rfind("}") + 1]
            return json.loads(s2)
        raise

def classify_text(text: str):
    """Return {"topic": str, "keywords": [..]} for a single llm_input string."""
    msg = chain.invoke({"text": text or ""})
    data = _safe_json_loads(msg.content)
    topic = (data.get("topic") or "").strip()
    keywords = data.get("keywords") or []
    if not isinstance(keywords, list):
        keywords = [str(keywords)]
    keywords = [str(k).strip() for k in keywords if str(k).strip()]
    return {"topic": topic, "keywords": keywords}

# -----------------------------
# 4) Classify a whole DataFrame using tqdm and llm_input
# -----------------------------
def classify_and_save_batches(
    df,
    llm_input_col="llm_input",
    save_path="classified_results.csv",
    batch_size=100,
    resume=True,
):
    """
    Classify posts using the 'llm_input' column and save results in batches.
    Writes a CSV that includes the original dataframe columns + topic + keywords.

    Parameters
    ----------
    df : pd.DataFrame
        Your dataframe (must include llm_input_col).
    llm_input_col : str
        Column name containing the preprocessed Title/Body text.
    save_path : str
        Where to write incremental CSV results.
    batch_size : int
        Number of rows to classify per checkpoint write.
    resume : bool
        If True and save_path exists, resume from the number of rows already saved.
    """
    assert llm_input_col in df.columns, f"Missing column: {llm_input_col}"

    # Keep a stable row id to match saved progress
    work = df.copy()
    work["_row_id"] = work.index  # stable id for this session
    work = work.reset_index(drop=True)

    # Determine starting offset if resuming
    start = 0
    write_header = True
    if resume and os.path.exists(save_path):
        try:
            # Only read the _row_id column to know how many rows are saved
            prev = pd.read_csv(save_path, usecols=["_row_id"])
            done = len(prev)
            if done > 0:
                start = done
                write_header = False
                print(f"Resuming from row {start} based on existing file: {save_path}")
        except Exception:
            # If we cannot parse the existing file, start from scratch
            print("Could not parse existing save file; starting from scratch.")
            start = 0
            write_header = True

    n = len(work)
    if start >= n:
        print("Nothing to do: all rows appear to be processed.")
        return pd.read_csv(save_path) if os.path.exists(save_path) else work

    # Loop in batches and save after each batch
    for i in tqdm(range(start, n, batch_size), desc="Classifying (batched)"):
        j = min(i + batch_size, n)
        batch = work.iloc[i:j].copy()

        topics, kw_lists = [], []
        for text in tqdm(batch[llm_input_col].fillna("").astype(str).tolist(),
                         leave=False, desc=f"Batch {i}:{j}"):
            try:
                result = classify_text(text)
                topics.append(result["topic"])
                kw_lists.append(result["keywords"])
            except Exception:
                topics.append("UNPARSEABLE")
                kw_lists.append([])

        batch["topic"] = topics
        batch["keywords"] = kw_lists

        # Write this batch to disk (append if not the first write)
        mode = "w" if write_header else "a"
        batch.to_csv(save_path, mode=mode, index=False, header=write_header)
        write_header = False  # subsequent writes will append

    # Return the full saved results (merge original columns with predictions)
    final = pd.read_csv(save_path)

    # Ensure full schema: if CSV lost dtypes, it's okay; keep as-is
    return final

In [32]:
# -----------------------------
# 5) Example usage
# -----------------------------
# df_ready  # your DataFrame shown in the screenshot (has 'llm_input')
df_classified = classify_and_save_batches(
    df_ready,
    llm_input_col="llm_input",
    save_path="logging_topics_2025.csv",
    batch_size=50,
    resume=True,
)
df_classified.head()

Resuming from row 114600 based on existing file: logging_topics_2025.csv


Batch 114800:114850:  36%|███▌      | 18/50 [00:22<00:40,  1.27s/it]


Batch 115000:115050:  76%|███████▌  | 38/50 [00:48<00:19,  1.61s/it]


Batch 115250:115300:  12%|█▏        | 6/50 [00:07<00:50,  1.14s/it]


Batch 115450:115500:  52%|█████▏    | 26/50 [00:30<00:29,  1.24s/it]


...

Batch 118500:118550:  92%|█████████▏| 46/50 [00:59<00:05,  1.47s/it]


Batch 118750:118800:  28%|██▊       | 14/50 [00:17<00:45,  1.27s/it]


Batch 118950:119000:  68%|██████▊   | 34/50 [00:43<00:20,  1.29s/it]


Classifying (batched): 100%|██████████| 91/91 [1:36:41<00:00, 63.75s/it]


,Title,Body,clean_title,clean_body,llm_input,_row_id,topic,keywords
0,log4net with ASP.NET MVC: nothing happens,<p>I am trying to use log4net with ASP.NET MVC...,log4net with ASP.NET MVC: nothing happens,I am trying to use log4net with ASP.NET MVC an...,Title:\nlog4net with ASP.NET MVC: nothing happ...,16,Logback and Application Configuration,"['log4net', 'ASP.NET MVC', 'configuration', 'l..."
1,How can I resolve the event log full error upo...,<p>I want to add a new connection for a SQL da...,How can I resolve the event log full error upo...,I want to add a new connection for a SQL datab...,Title:\nHow can I resolve the event log full e...,22,Event Logging and Monitoring,"['event log', 'full error', 'Visual C#', 'SQL ..."
2,Customizing NSLog Function on iPhone,<p>I know that it's possible to do method swiz...,Customizing NSLog Function on iPhone,I know that it's possible to do method swizzli...,Title:\nCustomizing NSLog Function on iPhone\n...,23,Custom Logging Frameworks,"['NSLog', 'method swizzling', 'Objective C', '..."
3,Why Does My Vector<PEVENTLOGRECORD> Mysterious...,<p>I am making a program that reads and stores...,Why Does My Vector Mysteriously Get Cleared?,I am making a program that reads and stores da...,Title:\nWhy Does My Vector Mysteriously Get Cl...,26,Error Handling and Debugging,"['EventLog', 'C++', 'vector', 'ReadEventLog', ..."
4,Use log4net in SDK,<p>I am providing an SDK using C#. To enable f...,Use log4net in SDK,I am providing an SDK using C#. To enable fiel...,Title:\nUse log4net in SDK\n\nBody:\nI am prov...,34,NLog and .NET Logging,"['log4net', 'C#', 'SDK', 'field debugging', 'c..."


In [33]:
df_classified

,Title,Body,clean_title,clean_body,llm_input,_row_id,topic,keywords
0,log4net with ASP.NET MVC: nothing happens,<p>I am trying to use log4net with ASP.NET MVC...,log4net with ASP.NET MVC: nothing happens,I am trying to use log4net with ASP.NET MVC an...,Title:\nlog4net with ASP.NET MVC: nothing happ...,16,Logback and Application Configuration,"['log4net', 'ASP.NET MVC', 'configuration', 'l..."
1,How can I resolve the event log full error upo...,<p>I want to add a new connection for a SQL da...,How can I resolve the event log full error upo...,I want to add a new connection for a SQL datab...,Title:\nHow can I resolve the event log full e...,22,Event Logging and Monitoring,"['event log', 'full error', 'Visual C#', 'SQL ..."
2,Customizing NSLog Function on iPhone,<p>I know that it's possible to do method swiz...,Customizing NSLog Function on iPhone,I know that it's possible to do method swizzli...,Title:\nCustomizing NSLog Function on iPhone\n...,23,Custom Logging Frameworks,"['NSLog', 'method swizzling', 'Objective C', '..."
3,Why Does My Vector<PEVENTLOGRECORD> Mysterious...,<p>I am making a program that reads and stores...,Why Does My Vector Mysteriously Get Cleared?,I am making a program that reads and stores da...,Title:\nWhy Does My Vector Mysteriously Get Cl...,26,Error Handling and Debugging,"['EventLog', 'C++', 'vector', 'ReadEventLog', ..."
4,Use log4net in SDK,<p>I am providing an SDK using C#. To enable f...,Use log4net in SDK,I am providing an SDK using C#. To enable fiel...,Title:\nUse log4net in SDK\n\nBody:\nI am prov...,34,NLog and .NET Logging,"['log4net', 'C#', 'SDK', 'field debugging', 'c..."
...,...,...,...,...,...,...,...,...
119119,why isn't my GCS-triggered Python Gen2 Cloud F...,<p>I am trying to trigger a simple Python 3.11...,why isn't my GCS-triggered Python Gen2 Cloud F...,I am trying to trigger a simple Python 3.11 Ge...,Title:\nwhy isn't my GCS-triggered Python Gen2...,215257,Error Handling and Debugging,"['GCS-triggered', 'Cloud Function', 'logging',..."
119120,How do I stop my app from auto-logging my Andr...,<p>I am testing authentication with a mobile a...,How do I stop my app from auto-logging my Andr...,I am testing authentication with a mobile app....,Title:\nHow do I stop my app from auto-logging...,215258,Error Handling and Debugging,"['authentication', 'Firebase', 'testing', 'ski..."
119121,LastLoginTime not present in Windows 11. Is th...,<p>I am working to write some simple powershel...,LastLoginTime not present in Windows 11. Is th...,I am working to write some simple powershell s...,Title:\nLastLoginTime not present in Windows 1...,215259,General Logging Practices,"['Windows 11', 'last login', 'local accounts',..."
119122,Why is my server not logging a login session?,<p>For some context: I'm trying to make a basi...,Why is my server not logging a login session?,For some context: I'm trying to make a basic l...,Title:\nWhy is my server not logging a login s...,215260,Error Handling and Debugging,"['login session', 'server not recognizing', 'N..."
